In [1]:
import pandas as pd
import subprocess
import glob
import numpy as np

In [3]:
# prior to this, train final model on 100% of data using script_2A_gnn_final_train.sh

# Part 1: Use final model on Broad800k library

In [ ]:
# use a small chunk of 1,000 compounds at a time for models to predict on
# can do less or more depending on your RAM
chunksize = 1000

# make directory for results to live
!mkdir broad800k_preds/

# preprocess data - SMILES and RDKit 2D normalized features
data = pd.read_csv('../data/broad800k.csv')
with np.load('../data/broad.npz') as ftsdata:
    ftsdata = ftsdata['features']

# chunkify the features and SMILES and save the pre-chunked features to files
for i in range(0, len(data), chunksize):
    smis = pd.DataFrame(data.iloc[i:i+chunksize,:])
    smis.to_csv('broad800k_preds/' + str(i) + '.csv', index = False)
    nps = ftsdata[i:i+chunksize,:]
    np.save('broad800k_preds/' + str(i) + '.npy', nps)

# to predict, use subprocess, parallel, or whatever you want!
for i in range(0, len(data), chunksize):
    print(i)
    activate_command = 'conda activate chemprop; '
    run_command = 'python predict.py --test_path ../nontoxic_stat_phase_killing_abx/broad800k_preds/' + str(i) + '.csv --checkpoint_dir ../nontoxic_stat_phase_killing_abx/models/finaltrain16/ --preds_path ../nontoxic_stat_phase_killing_abx/broad800k_preds/' + str(i) + '_preds.csv --features_path ../nontoxic_stat_phase_killing_abx/broad800k_preds/' + str(i) + '.npy --no_features_scaling --smiles_column smiles --gpu 0 --ensemble_variance'
    full_command = activate_command + run_command
    test = subprocess.run(full_command, cwd="../chemprop/", shell=True, capture_output=True)

In [ ]:
# now combine predictions across all chunks
chemprop = pd.read_csv('broad800k_preds/0_preds.csv')
for i in range(chunksize, len(data), chunksize):
    new = pd.read_csv('broad800k_preds/' + str(i) + '_preds.csv')
    chemprop = pd.concat([chemprop, new])
chemprop.to_csv('../out/broad800k_model_predictions_08_20_2021.csv', index = False)